<a href="https://colab.research.google.com/github/AtSourav/AE-w.-Bottleneck-Residual-Blocks/blob/main/AE_cifar10_residual_blocks_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow import math
import tensorflow_datasets as tfds
from keras import losses
from keras import layers
from keras import utils
from keras import metrics
from keras import backend as K
from keras import initializers

import os
import random
import matplotlib.pyplot as plt

In [2]:
! git clone https://github.com/AtSourav/AE-w.-Bottleneck-Residual-Blocks

%cd "/content/AE-w.-Bottleneck-Residual-Blocks"

import plotting_tools as plts
import Residual_blocks as resblock
import Bottleneck_residual_blocks as bresblock

Cloning into 'AE-w.-Bottleneck-Residual-Blocks'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 97 (delta 43), reused 55 (delta 17), pack-reused 0
Receiving objects: 100% (97/97), 49.26 KiB | 3.52 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/AE-w.-Bottleneck-Residual-Blocks


In [3]:
img_ht = 32
img_wd = 32

input_size = (img_ht,img_wd,3)
latent_dim = 256
batch_size = 128

initializer = initializers.HeNormal(seed=123)

In [4]:
ds = tfds.load('cifar10', split='train')
ds2= tfds.load('cifar10', split='test')

ds_to_np = list(ds.as_numpy_iterator())
ds2_to_np = list(ds2.as_numpy_iterator())

img_train = np.array([x['image'] for x in ds_to_np])
img_valid = np.array([x['image'] for x in ds2_to_np])

img_train = img_train/255
img_valid = img_valid/255

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteF7NRYD/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteF7NRYD/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [5]:
encoder_input = keras.Input(shape=input_size)

x = resblock.bridge_residual_conv2D_block(64, 2, 3, initializer, 'min', use_bn = 'False')(encoder_input)

# x = resblock.residual_conv2D_block(64, 1, 2, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(x)

#####################################################################################

# the default padding is 'valid'

x = resblock.bridge_residual_conv2D_block(128, 1, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 1, 1, initializer, 'min', use_bn = 'False')(x)

# x = resblock.residual_conv2D_block(128, 1, 1, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = resblock.bridge_residual_conv2D_block(256, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(256, 1, 1, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(256, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(256, 1, 1, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = resblock.bridge_residual_conv2D_block(512, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(512, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(512, 1, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(512, 1, 1, initializer, 'min', use_bn = 'False')(x)

#####################################################################################

x = layers.Flatten()(x)

x = layers.Dense(3*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*latent_dim)(x)
x = layers.ReLU()(x)


z = layers.Dense(latent_dim, name="z")(x)

encoder = keras.Model(encoder_input, z, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 bridge_residual_conv2d_blo  (None, 28, 28, 64)        39232     
 ck (bridge_residual_conv2D                                      
 _block)                                                         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 bridge_residual_conv2d_blo  (None, 13, 13, 128)       41728     
 ck_1 (bridge_residual_conv                                      
 2D_block)                                                       
                                                           

In [6]:
latent_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(2*latent_dim)(latent_input)
x = layers.ReLU()(x)

x = layers.Dense(3*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(4*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*2*1024)(x)
x = layers.ReLU()(x)

x = layers.Reshape((2,2,1024))(x)

x = resblock.bridge_residual_conv2Dtrans_block(1024, 1, 1, initializer, use_bn = 'False')(x)

##############################################################################################

x = resblock.bridge_residual_conv2Dtrans_block(512, 1, 1, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(512, 2, 2, initializer, use_bn = 'False')(x)

#############################################################################################

x = resblock.bridge_residual_conv2Dtrans_block(256, 1, 1, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(256, 2, 2, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(256, 1, 2, initializer, use_bn = 'False')(x)

#############################################################################################

x = layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear')(x)

#############################################################################################

x = resblock.bridge_residual_conv2Dtrans_block(128, 2, 2, initializer, use_bn = 'False')(x)

x = resblock.residual_conv2Dtrans_block(128, 2, 2, initializer, use_bn = 'False')(x)

############################################################################################

x = layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear')(x)

###########################################################################################

x = resblock.bridge_residual_conv2D_block(128, 2, 2, initializer, 'min', use_bn = 'False')(x)

x = resblock.residual_conv2D_block(128, 1, 1, initializer, 'min', use_bn = 'False')(x)

###########################################################################################

x = resblock.bridge_residual_conv2D_block(64, 2, 2, initializer, 'min', use_bn = 'False')(x)

###########################################################################################

decoder_output = layers.Conv2D(3, 1, activation='sigmoid', padding='valid', kernel_initializer=initializer)(x)

decoder = keras.Model(latent_input, decoder_output, name="decoder")
decoder.summary()


Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256)]             0         
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 re_lu_26 (ReLU)             (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 768)               393984    
                                                                 
 re_lu_27 (ReLU)             (None, 768)               0         
                                                                 
 dense_4 (Dense)             (None, 1024)              787456    
                                                                 
 re_lu_28 (ReLU)             (None, 1024)              0   

In [7]:
decoder_out = decoder(encoder(encoder_input))
AE = keras.Model(encoder_input, decoder_out, name='AE')

AE.summary()

Model: "AE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 encoder (Functional)        (None, 256)               8539328   
                                                                 
 decoder (Functional)        (None, 32, 32, 3)         12404995  
                                                                 
Total params: 20944323 (79.90 MB)
Trainable params: 20930499 (79.84 MB)
Non-trainable params: 13824 (54.00 KB)
_________________________________________________________________


In [ ]:
K.clear_session()

In [8]:
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.0001, decay_steps=2000, decay_rate=0.7)
#callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
optim = keras.optimizers.Adam(learning_rate = 0.0001)
AE.compile(optimizer=optim,loss='mse')
history = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=75)

Epoch 1/75
391/391 [==============================] - 153s 297ms/step - loss: 0.0412 - val_loss: 0.0398
Epoch 2/75
391/391 [==============================] - 108s 276ms/step - loss: 0.0270 - val_loss: 0.0277
Epoch 3/75
391/391 [==============================] - 108s 276ms/step - loss: 0.0235 - val_loss: 0.0265
Epoch 4/75
391/391 [==============================] - 108s 275ms/step - loss: 0.0212 - val_loss: 0.0232
Epoch 5/75
391/391 [==============================] - 103s 264ms/step - loss: 0.0195 - val_loss: 0.0207
Epoch 6/75
391/391 [==============================] - 108s 275ms/step - loss: 0.0186 - val_loss: 0.0201
Epoch 7/75
391/391 [==============================] - 108s 275ms/step - loss: 0.0179 - val_loss: 0.0197
Epoch 8/75
391/391 [==============================] - 108s 275ms/step - loss: 0.0172 - val_loss: 0.0184
Epoch 9/75
391/391 [==============================] - 108s 276ms/step - loss: 0.0164 - val_loss: 0.0179
Epoch 10/75
391/391 [==============================] - 108s 275m

In [9]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
optim = keras.optimizers.Adam(learning_rate = 0.00001)
AE.compile(optimizer=optim,loss='mse')
history2 = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=75, callbacks=[callback])

Epoch 1/75
391/391 [==============================] - 130s 266ms/step - loss: 0.0090 - val_loss: 0.0102
Epoch 2/75
391/391 [==============================] - 103s 264ms/step - loss: 0.0089 - val_loss: 0.0102
Epoch 3/75
391/391 [==============================] - 108s 275ms/step - loss: 0.0089 - val_loss: 0.0102
Epoch 4/75
391/391 [==============================] - 108s 275ms/step - loss: 0.0089 - val_loss: 0.0102
Epoch 5/75
391/391 [==============================] - 108s 275ms/step - loss: 0.0088 - val_loss: 0.0102
Epoch 6/75
391/391 [==============================] - 108s 276ms/step - loss: 0.0088 - val_loss: 0.0102
Epoch 7/75
391/391 [==============================] - 103s 264ms/step - loss: 0.0088 - val_loss: 0.0102
Epoch 8/75
391/391 [==============================] - 103s 264ms/step - loss: 0.0088 - val_loss: 0.0103
Epoch 9/75
391/391 [==============================] - 108s 277ms/step - loss: 0.0088 - val_loss: 0.0102
Epoch 10/75
391/391 [==============================] - 108s 276m

KeyboardInterrupt: 

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
optim = keras.optimizers.Adam(learning_rate = 0.000001)
AE.compile(optimizer=optim,loss='mse')
history3 = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=75, callbacks=[callback])

Epoch 1/75
391/391 [==============================] - 129s 273ms/step - loss: 0.0087 - val_loss: 0.0101
Epoch 2/75
391/391 [==============================] - 103s 262ms/step - loss: 0.0087 - val_loss: 0.0101
Epoch 3/75
391/391 [==============================] - 108s 276ms/step - loss: 0.0087 - val_loss: 0.0101
Epoch 4/75
391/391 [==============================] - 103s 264ms/step - loss: 0.0087 - val_loss: 0.0101
Epoch 5/75
391/391 [==============================] - 108s 276ms/step - loss: 0.0087 - val_loss: 0.0101
Epoch 6/75
391/391 [==============================] - 103s 264ms/step - loss: 0.0087 - val_loss: 0.0101
Epoch 7/75
391/391 [==============================] - 103s 263ms/step - loss: 0.0087 - val_loss: 0.0101
Epoch 8/75
391/391 [==============================] - 108s 276ms/step - loss: 0.0087 - val_loss: 0.0101
Epoch 9/75
157/391 [===========>..................] - ETA: 58s - loss: 0.0087